In [1]:
%pip install pyspark

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType, TimestampType
from pyspark.sql import functions as F
from pyspark.sql.functions import col,isnan,when,count,lit
from pyspark.ml.feature import Imputer
from pyspark.sql import Window
from pyspark.sql.functions import month,year,dayofmonth

In [4]:
spark = SparkSession.builder.appName('BigData_EDA').getOrCreate()
spark

In [5]:
# reading the CSV file top5 are the 5 crops i went for to explore more
filePath = '/content/drive/MyDrive/Colab_Notebooks/cropProduction.csv'
crop=pd.read_csv(filePath)
crop.head(5)

,Season,Region,Year,Tomatoes,Cucumber,Broad beans,Okra,Onion dry
0,Summer,Irbid,2017.0,11124.5,519.0,37.0,319.0,294.0
1,Summer,Irbid,2018.0,15540.4,10015.8,1455.0,1003.1,781.9
2,Summer,Irbid,2019.0,6292.4,1125.0,841.4,1808.7,5616.0
3,Summer,Irbid,2020.0,5122.7,554.6,1913.5,1428.2,6269.1
4,Summer,Irbid,2021.0,2968.9,3713.4,962.9,867.3,7770.8


In [6]:
crop.shape

(21, 8)

In [7]:
crop.dtypes

Season          object
Region          object
Year           float64
Tomatoes       float64
Cucumber       float64
Broad beans    float64
Okra           float64
Onion dry      float64
dtype: object

In [8]:
crop=crop.dropna()

# Visualizing Crop Production with Plotly

*   to notice trends, patterns on any sudden rises or drops
*   in which region is crop yield higher for crop X?
* does the seaon (weather) have an effect on crop yeild?
* does the sudden drop / increase be relative to an event in the given year?



In [9]:
custom = {
    'Winter': 'rgb(100, 149, 237)',
    'Summer': 'rgb(255, 99, 71)'
}
fig = px.bar(crop, x='Year',y='Tomatoes' ,color='Season', facet_col='Region' ,title="Yields for Tomato in both regions based on season 2017-2021", color_discrete_map= custom)
fig.show()

In [10]:
fig = px.bar(crop, x='Year',y='Onion dry' ,color='Season', facet_col='Region' ,title="Yields for Onion in both regions based on season 2017-2021",color_discrete_map= custom)
fig.show()

In [11]:
fig = px.bar(crop, x='Year',y='Okra' ,color='Season', facet_col='Region' ,title="Yields for Okra", color_discrete_map= custom)
fig.show()

# Visualising Historical Weather dataset
* can we find correlations between weather and crop yeilds in their corresponding years?
* does the weather in the region affect production of crops?

In [12]:
# this is the raw time series data
filePath2 ='/content/drive/MyDrive/Colab_Notebooks/irbidTimeSeries.csv'
irbid=pd.read_csv(filePath2)

In [13]:
irbid.head()

,Station,Date/Time(year),Air Dew Point,Air Temperature (OC),Humidity %,Manual Present Weather,Cloud Type,Clouds Cover (Okta),Cloud Cover %,Wind Direction (Degrees),Wind Speed (MPS),Wind Type
0,Irbid,2017-01-01T06:00:00,4,5,94,NaN,cumulus,2,25.0,NaN,0.0,calm
1,Irbid,2017-01-01T09:00:00,7,8,91,NaN,cumulus,2,25.0,300.0,3.0,normal
2,Irbid,2017-01-01T12:00:00,6,11,72,NaN,cumulus,3,38.0,250.0,3.0,normal
3,Irbid,2017-01-01T15:00:00,6,9,82,NaN,cumulus,2,25.0,NaN,0.0,calm
4,Irbid,2017-01-01T18:00:00,2,6,75,NaN,NaN,NaN,0.0,NaN,0.0,calm


In [14]:
irbid.dtypes

Station                      object
Date/Time(year)              object
Air Dew Point                object
Air Temperature (OC)         object
Humidity %                   object
Manual Present Weather       object
Cloud Type                   object
Clouds Cover (Okta)          object
Cloud Cover %               float64
Wind Direction (Degrees)    float64
Wind Speed (MPS)            float64
Wind Type                    object
dtype: object

In [15]:
irbid.shape

(8043, 12)

In [16]:
irbid.describe()

,Cloud Cover %,Wind Direction (Degrees),Wind Speed (MPS)
count,8042.00000,5722.000000,8040.000000
mean,2.25230,234.192590,1.728731
std,10.34069,74.623239,1.451002
min,0.00000,10.000000,0.000000
25%,0.00000,210.000000,0.000000
50%,0.00000,260.000000,2.000000
75%,0.00000,280.000000,3.000000
max,100.00000,360.000000,21.000000


In [17]:
irbid.nunique()

Station                        1
Date/Time(year)             8043
Air Dew Point                 42
Air Temperature (OC)          45
Humidity %                    91
Manual Present Weather        13
Cloud Type                     6
Clouds Cover (Okta)            9
Cloud Cover %                  9
Wind Direction (Degrees)      36
Wind Speed (MPS)              12
Wind Type                      2
dtype: int64

In [18]:
irbid.isna().sum()

Station                        0
Date/Time(year)                0
Air Dew Point                  0
Air Temperature (OC)           0
Humidity %                     0
Manual Present Weather      8001
Cloud Type                  7641
Clouds Cover (Okta)         7643
Cloud Cover %                  1
Wind Direction (Degrees)    2321
Wind Speed (MPS)               3
Wind Type                      1
dtype: int64

In [19]:
fig = px.line(irbid, x='Date/Time(year)', y="Air Temperature (OC)")
fig.show()

In [20]:
fig = px.histogram(irbid, x='Date/Time(year)', y="Wind Speed (MPS)")
fig.show()

most data is null or showing irregular readings, the following CSV was extracted from IrbidProduction.ipynb where the time stamp was grouped using PySpark withColumn()

In [21]:
filePath3='/content/drive/MyDrive/Colab_Notebooks/irbidViz.csv'
irbid2=pd.read_csv(filePath3)
irbid2.head(10)

,day,month,year,Station,avg(Air Dew Point),avg(Air Temperature (OC)),avg(Humidity %),avg(Clouds Cover (Okta)),avg(Clouds Cover %),avg(Wind Direction (Degrees)),avg(Wind Speed (MPS)),season
0,1,1,2017,Irbid,5.00,7.800000,82.800000,2.250000,22.60,275.0,1.2,Winter
1,2,1,2017,Irbid,3.60,8.000000,75.800000,3.200000,40.40,210.0,1.8,Winter
2,3,1,2017,Irbid,6.00,8.333333,84.333333,3.333333,42.00,220.0,3.0,Winter
3,4,1,2017,Irbid,2.80,9.200000,66.000000,2.600000,32.60,NaN,0.0,Winter
4,5,1,2017,Irbid,-0.40,12.000000,48.000000,3.800000,47.80,NaN,0.0,Winter
5,6,1,2017,Irbid,-3.75,10.200000,35.750000,NaN,0.00,120.0,1.6,Winter
6,7,1,2017,Irbid,-1.20,10.200000,46.000000,1.500000,7.60,240.0,1.2,Winter
7,8,1,2017,Irbid,-2.80,7.800000,47.600000,2.750000,27.80,245.0,0.8,Winter
8,9,1,2017,Irbid,-4.60,6.800000,44.200000,3.400000,42.60,227.5,4.8,Winter
9,10,1,2017,Irbid,-5.25,8.250000,40.500000,3.333333,31.25,200.0,3.0,Winter


In [22]:
irbid2.isna().sum()

day                                 0
month                               0
year                                0
Station                             0
avg(Air Dew Point)               1819
avg(Air Temperature (OC))           0
avg(Humidity %)                  1819
avg(Clouds Cover (Okta))         1918
avg(Clouds Cover %)                 0
avg(Wind Direction (Degrees))     136
avg(Wind Speed (MPS))               0
season                              0
dtype: int64

In [23]:
irbid2.shape

(2040, 12)

In [24]:
irbid2.describe()

,day,month,year,avg(Air Dew Point),avg(Air Temperature (OC)),avg(Humidity %),avg(Clouds Cover (Okta)),avg(Clouds Cover %),avg(Wind Direction (Degrees)),avg(Wind Speed (MPS))
count,2040.000000,2040.000000,2040.000000,221.000000,2040.000000,221.000000,122.000000,2040.000000,1904.000000,2040.000000
mean,15.640686,6.179412,2019.694118,7.212217,19.122737,50.414329,3.461066,1.872574,233.440345,1.707272
std,8.790394,3.462426,1.696306,6.294328,7.060516,17.595573,0.856956,8.572866,65.219563,1.017803
min,1.000000,1.000000,2017.000000,-10.200000,2.000000,17.200000,1.500000,0.000000,10.000000,0.000000
25%,8.000000,3.000000,2018.000000,3.600000,12.575000,36.000000,3.000000,0.000000,203.333333,1.000000
50%,15.500000,6.000000,2020.000000,7.000000,20.000000,49.400000,3.400000,0.000000,255.000000,1.666667
75%,23.000000,9.000000,2021.000000,12.000000,25.000000,63.600000,4.000000,0.000000,275.000000,2.297619
max,31.000000,12.000000,2023.000000,22.000000,42.000000,96.000000,6.250000,75.400000,360.000000,12.500000


In [25]:
irbid2 = irbid2.drop(irbid2[irbid2['year'] == 2023].index)

In [26]:
fig = px.histogram(irbid2, x='avg(Air Temperature (OC))', title='Histogram of Air Temperature')
fig.show()

In [27]:
avg_temp_by_month = irbid2.groupby(['year', 'month'])['avg(Air Temperature (OC))'].mean().reset_index()

fig = px.line(avg_temp_by_month, x='month', y='avg(Air Temperature (OC))', color='year',
              title='Average Air Temperature Across Months for Each Year')
fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Average Air Temperature (OC)'))
fig.show()

In [28]:
avg_wind_by_month = irbid2.groupby(['year', 'month'])['avg(Wind Speed (MPS))'].mean().reset_index()

fig = px.line(avg_wind_by_month, x='month', y='avg(Wind Speed (MPS))', color='year',
              title='Average Wind Speeds Across Months for Each Year')
fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Average Wind Speed (MPS)'))
fig.show()

In [29]:
avg_dir_by_month = irbid2.groupby(['year', 'month'])['avg(Wind Direction (Degrees))'].mean().reset_index()

fig = px.line(avg_dir_by_month, x='month', y='avg(Wind Direction (Degrees))', color='year',
              title='Average Wind Directions Across Months for Each Year')
fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Average Wind Direction'))
fig.show()

## Any extreme weather conditions throughout the seasons?

In [30]:
fig = px.box(irbid2, x="year", y="avg(Air Temperature (OC))", color='season', points= 'outliers')
fig.show()
# the season column allows to map with crop production data

In [31]:
fig = px.box(irbid2, x="year", y="avg(Wind Speed (MPS))", color='season', points= 'outliers')
fig.show()

In [32]:
fig = px.box(irbid2, x="year", y="avg(Wind Direction (Degrees))", color='season', points= 'outliers')
fig.show()

#Correlation between crop productions and historical weather

but first, we need to merge the weather data with the crop data, the strategy here is to see a general correlation with extreme weather values

In [33]:
crop_irbid=crop[crop['Region'] != 'Ghor Safi']
crop_irbid.head()

,Season,Region,Year,Tomatoes,Cucumber,Broad beans,Okra,Onion dry
0,Summer,Irbid,2017.0,11124.5,519.0,37.0,319.0,294.0
1,Summer,Irbid,2018.0,15540.4,10015.8,1455.0,1003.1,781.9
2,Summer,Irbid,2019.0,6292.4,1125.0,841.4,1808.7,5616.0
3,Summer,Irbid,2020.0,5122.7,554.6,1913.5,1428.2,6269.1
4,Summer,Irbid,2021.0,2968.9,3713.4,962.9,867.3,7770.8


In [34]:
#using spark to group by year to find avg for that year and max (extreme) values
irbidSchema = StructType([

 StructField('Day', IntegerType(), True),
 StructField('Month',IntegerType(),True),
 StructField('Year', IntegerType(),True),
 StructField('Region', StringType(),True),
 StructField('airdewpoint', FloatType(),True),
 StructField('temperature', FloatType(),True),
 StructField('humidity',FloatType(),True),
 StructField('cloudcover(okta)', FloatType(),True),
 StructField('cloudcover(%)', FloatType(),True),
 StructField('winddir', FloatType(), True),
 StructField('windspeed', FloatType(), True),
 StructField('Season', StringType(), True)
 ])
irbidDf = spark.read.csv('/content/drive/MyDrive/Colab_Notebooks/vslab/final/irbidForViz.csv', header=True, schema=irbidSchema)

In [35]:
from pyspark.sql.functions import max

irbid_max = irbidDf.groupBy("Year", "Season", "Region").agg(
    max("airdewpoint").alias("max_airdewpoint"),
    max("temperature").alias("max_temperature"),
    max("humidity").alias("max_humidity"),
    max("cloudcover(okta)").alias("max_cloudcover_okta"),
    max("cloudcover(%)").alias("max_cloudcover_percent"),
    max("winddir").alias("max_winddir"),
    max("windspeed").alias("max_windspeed")
)

In [36]:
irbid_max.show()

+----+------+------+---------------+---------------+------------+-------------------+----------------------+-----------+-------------+
|Year|Season|Region|max_airdewpoint|max_temperature|max_humidity|max_cloudcover_okta|max_cloudcover_percent|max_winddir|max_windspeed|
+----+------+------+---------------+---------------+------------+-------------------+----------------------+-----------+-------------+
|2019|Summer| Irbid|           NULL|           36.0|        NULL|               NULL|                   0.5|      350.0|          4.0|
|2019|Winter| Irbid|            7.0|      30.333334|        43.0|               NULL|                   0.0|      350.0|         4.25|
|2022|Winter| Irbid|           12.0|           28.0|        83.0|                5.0|                 15.75|      350.0|    5.3333335|
|2017|Summer| Irbid|           22.0|           35.0|       74.75|                5.0|                  62.8|      340.0|          4.4|
|2020|Summer| Irbid|           NULL|           42.0|   

In [37]:
from pyspark.sql.functions import mean

irbid_mean = irbidDf.groupBy("Year", "Season", "Region").agg(
    mean("airdewpoint").alias("mean_airdewpoint"),
    mean("temperature").alias("mean_temperature"),
    mean("humidity").alias("mean_humidity"),
    mean("cloudcover(okta)").alias("mean_cloudcover_okta"),
    mean("cloudcover(%)").alias("mean_cloudcover_percent"),
    mean("winddir").alias("mean_winddir"),
    mean("windspeed").alias("mean_windspeed")
)

In [38]:
irbid_mean.show()

+----+------+------+------------------+------------------+-----------------+--------------------+-----------------------+------------------+------------------+
|Year|Season|Region|  mean_airdewpoint|  mean_temperature|    mean_humidity|mean_cloudcover_okta|mean_cloudcover_percent|      mean_winddir|    mean_windspeed|
+----+------+------+------------------+------------------+-----------------+--------------------+-----------------------+------------------+------------------+
|2019|Summer| Irbid|              NULL| 24.18362575664855|             NULL|                NULL|   0.002923976608187...|262.95535714285717|1.6798245636691824|
|2019|Winter| Irbid|               7.0|14.059668506706616|             43.0|                NULL|                    0.0| 203.4428571864537|1.5726519335699345|
|2022|Winter| Irbid|              12.0|13.340218240684933|             83.0|                 5.0|                 0.0875|206.25598566949705|1.7962367742425867|
|2017|Summer| Irbid|10.784426227944795|2

In [39]:
irbid_meanP=irbid_mean.toPandas()

In [40]:
irbid_meanP.head()

,Year,Season,Region,mean_airdewpoint,mean_temperature,mean_humidity,mean_cloudcover_okta,mean_cloudcover_percent,mean_winddir,mean_windspeed
0,2019,Summer,Irbid,NaN,24.183626,NaN,NaN,0.002924,262.955357,1.679825
1,2019,Winter,Irbid,7.000000,14.059669,43.000000,NaN,0.000000,203.442857,1.572652
2,2022,Winter,Irbid,12.000000,13.340218,83.000000,5.000000,0.087500,206.255986,1.796237
3,2017,Summer,Irbid,10.784426,25.398333,44.201366,3.330952,10.164615,266.381782,1.134744
4,2020,Summer,Irbid,NaN,23.650803,NaN,NaN,0.000000,258.132697,1.682329


In [41]:
irbid_maxP=irbid_max.toPandas()

In [42]:
irbid_maxP.head()

,Year,Season,Region,max_airdewpoint,max_temperature,max_humidity,max_cloudcover_okta,max_cloudcover_percent,max_winddir,max_windspeed
0,2019,Summer,Irbid,NaN,36.000000,NaN,NaN,0.500000,350.0,4.000000
1,2019,Winter,Irbid,7.0,30.333334,43.00,NaN,0.000000,350.0,4.250000
2,2022,Winter,Irbid,12.0,28.000000,83.00,5.0,15.750000,350.0,5.333333
3,2017,Summer,Irbid,22.0,35.000000,74.75,5.0,62.799999,340.0,4.400000
4,2020,Summer,Irbid,NaN,42.000000,NaN,NaN,0.000000,360.0,4.000000


basically here i extracted the extree values via Pyspark then converted to pandas to visualise correlations

## General Correlations with Irbid Weather

In [43]:
correlation_matrix = irbid_meanP.corrwith(crop_irbid['Okra'])
df_corr = pd.DataFrame(correlation_matrix, columns=['Correlation'])
df_corr['Abs_Correlation'] = df_corr['Correlation'].abs()
df_corr = df_corr.sort_values(by='Abs_Correlation', ascending=False)

fig = px.bar(df_corr, x=df_corr.index, y='Correlation',
             title='Correlation between Mean Irbid Weather Columns and Okra Production',
             labels={'x': 'Irbid Weather Columns', 'Correlation': 'Correlation with Okra Production'},
             color='Correlation', color_continuous_scale='RdBu')
fig.update_layout(xaxis=dict(tickangle=45))
fig.show()


<ipython-input-43-f2c76d3a1474>:1: FutureWarning:

The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [44]:
correlation_matrix = irbid_meanP.corrwith(crop_irbid['Onion dry'])
df_corr = pd.DataFrame(correlation_matrix, columns=['Correlation'])
df_corr['Abs_Correlation'] = df_corr['Correlation'].abs()
df_corr = df_corr.sort_values(by='Abs_Correlation', ascending=False)

fig = px.bar(df_corr, x=df_corr.index, y='Correlation',
             title='Correlation between Mean Irbid Weather Columns and Onion Production',
             labels={'x': 'Irbid Weather Columns', 'Correlation': 'Correlation with Onion Production'},
             color='Correlation', color_continuous_scale='RdBu')
fig.update_layout(xaxis=dict(tickangle=45))
fig.show()


<ipython-input-44-74c36c7eae2a>:1: FutureWarning:

The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [45]:
correlation_matrix = irbid_meanP.corrwith(crop_irbid['Tomatoes'])
df_corr = pd.DataFrame(correlation_matrix, columns=['Correlation'])
df_corr['Abs_Correlation'] = df_corr['Correlation'].abs()
df_corr = df_corr.sort_values(by='Abs_Correlation', ascending=False)

fig = px.bar(df_corr, x=df_corr.index, y='Correlation',
             title='Correlation between Mean Irbid Weather Columns and Tomatoes Production',
             labels={'x': 'Irbid Weather Columns', 'Correlation': 'Correlation with Tomatoes Production'},
             color='Correlation', color_continuous_scale='RdBu')
fig.update_layout(xaxis=dict(tickangle=45))
fig.show()

<ipython-input-45-fbc1a673984d>:1: FutureWarning:

The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



### Confusion Matrices

In [46]:
merged_irbid_mean = pd.merge(crop_irbid, irbid_meanP, on=['Year', 'Season','Region'])

In [47]:
corr = merged_irbid_mean.corr();

<ipython-input-47-ac043854ba6b>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [48]:
fig = px.imshow(corr, text_auto=True, aspect="auto", color_continuous_scale='Viridis')
fig.show()
# Correlations with mean / avg values of weather

In [49]:
merged_irbid_max = pd.merge(crop_irbid, irbid_maxP, on=['Year', 'Season','Region'])
corr_max = merged_irbid_max.corr()
fig = px.imshow(corr, text_auto=True, aspect="auto")
fig.show()
# Correlations with Max / extreme values of weather

<ipython-input-49-c41a325a67da>:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.

